In [1]:
import os
import sys
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator
from pathlib import Path
import os
import pickle
import torch
import numpy as np
import soundfile as sf

from math import ceil
from model_vc import Generator
import signal
import soundfile as sf
from scipy import signal
from scipy.signal import get_window
from librosa.filters import mel
from collections import OrderedDict


from numpy.random import RandomState

from model_bl import D_VECTOR
from synthesis import build_model
from synthesis import wavegen

sys.path.append("hifi_gan")
import inference_e2e_mod

len_crop = 128
device = 'cuda:0'


def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad


device = 'cuda:0'


In [2]:
def pySTFT(x, fft_length=1024, hop_length=256):

    x = np.pad(x, int(fft_length//2), mode='reflect')
    
    noverlap = fft_length - hop_length
    shape = x.shape[:-1]+((x.shape[-1]-noverlap)//hop_length, fft_length)
    strides = x.strides[:-1]+(hop_length*x.strides[-1], x.strides[-1])
    result = np.lib.stride_tricks.as_strided(x, shape=shape,
                                             strides=strides)
    
    fft_window = get_window('hann', fft_length, fftbins=True)
    result = np.fft.rfft(fft_window * result, n=fft_length).T
    
    return np.abs(result)    
    
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

    
mel_basis = mel(16000, 1024, fmin=90, fmax=7600, n_mels=80).T
min_level = np.exp(-100 / 20 * np.log(10))
b, a = butter_highpass(30, 16000, order=5)


def convert(filename):
    prng = RandomState(1234)#int(filename[1:])) 
    x, fs = sf.read(filename)
    # Remove drifting noise
    y = signal.filtfilt(b, a, x)
    # Ddd a little random noise for model roubstness
    wav = y * 0.96 + (prng.rand(y.shape[0])-0.5)*1e-06
    # Compute spect
    D = pySTFT(wav).T
    # Convert to mel and normalize
    D_mel = np.dot(D, mel_basis)
    D_db = 20 * np.log10(np.maximum(min_level, D_mel)) - 16
    S = np.clip((D_db + 100) / 100, 0, 1)    
    return np.array(S.astype(np.float32))
    # save spect    
    # np.save(os.path.join(targetDir, subdir, fileName[:-4]),
    #         S.astype(np.float32), allow_pickle=False)    


# Load models

In [3]:
C = D_VECTOR(dim_input=80, dim_cell=768, dim_emb=256).eval().cuda()
c_checkpoint = torch.load('3000000-BL.ckpt')
new_state_dict = OrderedDict()


In [4]:

G = Generator(32,256,512,32).eval().to(device)

g_checkpoint = torch.load('autovc.ckpt', map_location=device)
G.load_state_dict(g_checkpoint['model'])


<All keys matched successfully>

In [5]:

spect_vc = pickle.load(open('results.pkl', 'rb'))
device = torch.device("cuda")
model = build_model().to(device)
checkpoint = torch.load("checkpoint_step001000000_ema.pth")
model.load_state_dict(checkpoint["state_dict"])


<All keys matched successfully>

# data

In [11]:
metadata = pickle.load(open('metadata.pkl', "rb"))


# target embedding
embs = []
libritts_speaker_dir = Path("/home/shacharm/Projects/ug/data/LibriTTS/train-clean-100/19/198")

libritts_speaker_dir = Path("/home/shacharm/Projects/ug/data/kids_speech/wav/")
for wav_file in libritts_speaker_dir.glob("abm*.wav"):
    melspec = convert(str(wav_file))

    if melspec.shape[0] < len_crop:
        continue
    left = np.random.randint(0, melspec.shape[0]-len_crop)

    melsp = torch.from_numpy(melspec[np.newaxis, left:left+len_crop, :]).cuda()
    emb = C(melsp)
    embs.append(emb.detach().squeeze().cpu().numpy())     

trg_emb = np.mean(embs, axis=0)
trg_emb = torch.from_numpy(trg_emb[np.newaxis, :]).to(device)

/home/shacharm/.pyenv/versions/3.7.9/envs/ug/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/shacharm/.pyenv/versions/3.7.9/envs/ug/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


IndexError: invalid index to scalar variable.

In [10]:
spect_vc = []

for sbmt_i in metadata:
    x_org = sbmt_i[2]
    x_org, len_pad = pad_seq(x_org)
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
    
    for sbmt_j in metadata:
                   
        if False:
            emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        else:  
            emb_trg = trg_emb
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        trg_name = '{}x{}'.format(sbmt_i[0], sbmt_j[0])
        spect_vc.append( (trg_name, uttr_trg) )
        waveform = wavegen(model, c=uttr_trg)   
        sf.write('out/' + trg_name+'.wav', waveform, 16000, 'PCM_24')        
        
with open('results.pkl', 'wb') as handle:
    pickle.dump(spect_vc, handle)          

 28%|██▊       | 6394/22784 [00:58<02:30, 109.07it/s]


KeyboardInterrupt: 

In [10]:
H = inference_e2e_mod.HifiEvaluator('g_03280000', device=device)

AttributeError: 'NoneType' object has no attribute 'resblock_kernel_sizes'

In [18]:
# librosa.output.write_wav(name+'.wav', waveform, sr=16000)



In [19]:
name

'p225xp225'